In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 1000) 
pd.set_option('display.max_rows', 100)   

import plotly
#import pyperclip
import plotly.graph_objs as go

import umap
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

display(plotly.__version__)

# from peptide_fragmentor.peptide_fragmentor import PeptideFragment0r
# PROTON = 1.00727646677
# offline.init_notebook_mode(connected=True)
# import matplotlib
# import matplotlib.pyplot as plt
# import seaborn as sns
# import glob
# import os
# import json

# import numpy as np
# import pymzml
# import pyqms
# import plotly.offline as offline

In [ ]:
peptide_forest_output = "../01Apr_E50.csv"
local_importance_output = "../01Apr_E50.csv_local_importance.csv"

In [ ]:
df = pd.read_csv(peptide_forest_output)
df.shape

In [ ]:
df.head()

In [ ]:
local_importance = pd.read_csv(local_importance_output, index_col=0)
local_importance.shape

In [ ]:
local_importance.head()

In [ ]:
local_importance.describe()

In [ ]:
# subset = local_importance.sample(frac=0.1)
# subset.index

In [ ]:
subset = df[
#     (df['q-value_mascot_1_0_0'] < 0.01) |
#     (df['q-value_omssa_2_1_9'] < 0.01) |
#     (df['q-value_msgfplus_v2018_06_28'] < 0.01) |
#     (df['q-value_xtandem_vengeance'] < 0.01) |
#     (df['q-value_msfragger_20190222'] < 0.01) |
    (df['q-value_RF-reg'] < 0.01)
]
subset.index

In [ ]:
df.loc[subset.index, 'Spectrum ID'].nunique()

In [ ]:
embedding = umap.UMAP(n_neighbors=7).fit_transform(local_importance.loc[subset.index,:].values)
# embedding = umap.UMAP().fit_transform(local_importance.loc[subset.index,:].values)



In [ ]:
def copy_to_clipboard(trace, points, selector):
    pyperclip.copy(points)

In [ ]:
def draw_umap(df, embedding, c):
    data = go.Scattergl(
        x=embedding[:, 0], y=embedding[:, 1],
        mode='markers',
        marker=dict(
            size=3,
            color=df.loc[subset.index, c],
            colorscale='rdylgn',
            showscale=True
        ),
        hovertext = "Q-values<br />" + \
        "Mascot:    " + df.loc[subset.index, "q-value_mascot_1_0_0"].astype(str) + "<br />" + \
        "OMSSA:     " + df.loc[subset.index, "q-value_omssa_2_1_9"].astype(str) + "<br />" + \
        "MSGF+:     " + df.loc[subset.index, "q-value_msgfplus_v2018_06_28"].astype(str) + "<br />" + \
        "X!Tandem:  " + df.loc[subset.index, "q-value_xtandem_vengeance"].astype(str) + "<br />" + \
        "MSFragger: " + df.loc[subset.index, "q-value_msfragger_20190222"].astype(str) + "<br />" + \
        "RF:        " + df.loc[subset.index, "q-value_RF-reg"].astype(str) + "<br />" + \
        "Spectrum ID: " + df.loc[subset.index, 'Spectrum ID'].astype(str) + "<br />" + \
        f"{c}:"  + df.loc[subset.index, c].astype(str)
#         "e1: " + embedding[:, 0] + "<br />" + \
#         "e2: " + embedding[:, 1]
        ,
        hoverinfo = "text"
    )
    fig = go.Figure(data=data)
    fig.update_layout(template="plotly_dark", title=c, height=800, width=800)
    return fig

In [ ]:
df.columns

In [ ]:
fig = draw_umap(df, embedding,'q-value_msfragger_20190222')
fig.show()

In [ ]:
fig = draw_umap(df, embedding,'q-value_omssa_2_1_9')
fig.show()

In [ ]:
fig = draw_umap(df, embedding,'q-value_mascot_1_0_0')
fig.show()

In [ ]:
fig = draw_umap(df, embedding,'q-value_msfragger_20190222')
fig.show()

In [ ]:
fig = draw_umap(df, embedding,'q-value_msfragger_20190222')
fig.show()

In [ ]:
fig = draw_umap(df, embedding,'delta_score_2_msgfplus_v2018_06_28')
fig.show()

In [ ]:
fig = draw_umap(df, embedding,'delta_score_2_msfragger_20190222')
fig.show()

In [ ]:
clusters = [
    [(3, 19.5), (5, 17.5)]
]

In [ ]:
for (top_left_x, top_left_y), (bottom_right_x, bottom_right_y) in clusters:
    masked_edf = edf[
        (edf[0] > top_left_x) & \
        (edf[0] < bottom_right_x) & \
        (edf[1] < top_left_y) & \
        (edf[1] > bottom_right_y) 
    ]
    display(local_importance.loc[masked_edf.index, :].describe())

In [ ]:
local_importance.describe()

In [ ]:
masked_edf.head()

In [ ]:
edf= pd.DataFrame(embedding)

In [ ]:
edf.head()

In [ ]:
fig = draw_umap(df, embedding, 'Accuracy (ppm)')
fig.show()

In [ ]:
col = "PepLen"
# eng = "msfragger_20190222"
fig = plotly.subplots.make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Histogram(
        x= df[(~df['Is decoy'])][col],
        name="target"
    )
)

# Overlay both histograms
fig.update_layout(
    barmode='overlay',
    title=f"Distribution of {col}"
)
fig.update_xaxes(title=f"{col}")
fig.update_yaxes(title="Frequency")
# fig.update_yaxes(title="q-value", secondary_y=True)
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.45)
fig.show()

In [ ]:
for c in df.columns:
    print(c)
#     if 'q-value' not in c:
#         continue
    if c in ['Sequence', 'Modifications', 'Is decoy', 'Protein ID',
       'Score_processed_mascot_1_0_0',
       'Score_processed_msfragger_20190222',
       'Score_processed_msgfplus_v2018_06_28', 'Score_processed_omssa_2_1_9',
       'Score_processed_xtandem_vengeance', 
       'Score_processed_RF-reg', 
       'top_target_mascot_1_0_0', 'top_target_msfragger_20190222',
       'top_target_msgfplus_v2018_06_28', 'top_target_omssa_2_1_9',
       'top_target_xtandem_vengeance', 'top_target_RF-reg']:
        continue
    
    fig.show()

In [ ]:
copy_to_clipboard

In [ ]:
plotly_color_scales = ['aggrnyl', 'agsunset', 'algae', 'amp', 'armyrose', 'balance',
             'blackbody', 'bluered', 'blues', 'blugrn', 'bluyl', 'brbg',
             'brwnyl', 'bugn', 'bupu', 'burg', 'burgyl', 'cividis', 'curl',
             'darkmint', 'deep', 'delta', 'dense', 'earth', 'edge', 'electric',
             'emrld', 'fall', 'geyser', 'gnbu', 'gray', 'greens', 'greys',
             'haline', 'hot', 'hsv', 'ice', 'icefire', 'inferno', 'jet',
             'magenta', 'magma', 'matter', 'mint', 'mrybm', 'mygbm', 'oranges',
             'orrd', 'oryel', 'peach', 'phase', 'picnic', 'pinkyl', 'piyg',
             'plasma', 'plotly3', 'portland', 'prgn', 'pubu', 'pubugn', 'puor',
             'purd', 'purp', 'purples', 'purpor', 'rainbow', 'rdbu', 'rdgy',
             'rdpu', 'rdylbu', 'rdylgn', 'redor', 'reds', 'solar', 'spectral',
             'speed', 'sunset', 'sunsetdark', 'teal', 'tealgrn', 'tealrose',
             'tempo', 'temps', 'thermal', 'tropic', 'turbid', 'twilight',
             'viridis', 'ylgn', 'ylgnbu', 'ylorbr', 'ylorrd']

In [ ]:
# for c_column in df.columns:
#     if c_column in ['Sequence', 'Modifications', 'Protein ID']:
#         continue
#     fig, ax = plt.subplots(figsize=(20,20))
#     sns.scatterplot(
#         ax = ax,
#         x=embedding[:, 0], 
#         y=embedding[:, 1], 
#         hue=df.loc[subset.index, c_column],
#         s=2,
#         alpha=1,
#         palette="YlOrRd" if 'score' in c_column else "Dark2",
#         linewidth=0,
#     )
    
#     plt.legend( bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
#     plt.show()
#     break
# # """Accent, Accent_r, Blues, Blues_r, BrBG, BrBG_r, BuGn, BuGn_r, BuPu, BuPu_r, CMRmap, CMRmap_r, Dark2, Dark2_r, GnBu, GnBu_r, Greens, Greens_r, Greys, Greys_r, OrRd, OrRd_r, Oranges, Oranges_r, PRGn, PRGn_r, Paired, Paired_r, Pastel1, Pastel1_r, Pastel2, Pastel2_r, PiYG, PiYG_r, PuBu, PuBuGn, PuBuGn_r, PuBu_r, PuOr, PuOr_r, PuRd, PuRd_r, Purples, Purples_r, RdBu, RdBu_r, RdGy, RdGy_r, RdPu, RdPu_r, RdYlBu, RdYlBu_r, RdYlGn, RdYlGn_r, Reds, Reds_r, Set1, Set1_r, Set2, Set2_r, Set3, Set3_r, Spectral, Spectral_r, Wistia, Wistia_r, YlGn, YlGnBu, YlGnBu_r, YlGn_r, YlOrBr, YlOrBr_r, YlOrRd, YlOrRd_r, afmhot, afmhot_r, autumn, autumn_r, binary, binary_r, bone, bone_r, brg, brg_r, bwr, bwr_r, cividis, cividis_r, cool, cool_r, coolwarm, coolwarm_r, copper, copper_r, cubehelix, cubehelix_r, flag, flag_r, gist_earth, gist_earth_r, gist_gray, gist_gray_r, gist_heat, gist_heat_r, gist_ncar, gist_ncar_r, gist_rainbow, gist_rainbow_r, gist_stern, gist_stern_r, gist_yarg, gist_yarg_r, gnuplot, gnuplot2, gnuplot2_r, gnuplot_r, gray, gray_r, hot, hot_r, hsv, hsv_r, icefire, icefire_r, inferno, inferno_r, jet, jet_r, magma, magma_r, mako, mako_r, nipy_spectral, nipy_spectral_r, ocean, ocean_r, pink, pink_r, plasma, plasma_r, prism, prism_r, rainbow, rainbow_r, rocket, rocket_r, seismic, seismic_r, spring, spring_r, summer, summer_r, tab10, tab10_r, tab20, tab20_r, tab20b, tab20b_r, tab20c, tab20c_r, terrain, terrain_r, twilight, twilight_r, twilight_shifted, twilight_shifted_r, viridis, viridis_r, vlag, vlag_r, winter, winter_r"""


In [ ]:
# plt.scatter(
#     embedding[:, 0], 
#     embedding[:, 1], 
#     s=1, 
# #     c=df.loc[subset.index, 'Is decoy']
# #     c=df.loc[subset.index, 'Score_processed_xtandem_vengeance']
# )

In [ ]:
# c_column = 'rank_mascot_1_0_0'
# for c_column in df.columns:
#     if c_column in ['Sequence', 'Modifications', 'Protein ID']:
#         continue
#     plt.scatter(
#         embedding[:, 0], 
#         embedding[:, 1], 
#         s=1, 
#         c=df.loc[subset.index, c_column]
#     )


In [ ]:
# fig, ax = plt.subplots()
# ax.scatter(
# embedding[:, 0], 
#     embedding[:, 1], 
#     s=1, 
#     c=df.loc[subset.index, c_column]
#     #     x, y, c=color, s=scale, label=color,
# #                alpha=0.3, edgecolors='none'
# )

# ax.legend()
# ax.grid(True)
# plt.show()